### Sorting and visualizing UniPept lowest common ancestor analyses in Trocas7

#### Beginning with: LCA peptides in csvs

#### Goal: spectral abundance-corrected (NAAF) taxonomic peptide compositions at the 4 stations before and after 24hrs

## Issue: the `join` commands at the end for diatom and bacterial peptides need to be run independently from one another. To do this, run the diatom command, restart the kernal, then run the bacterial. 

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [3]:
cd /home/millieginty/Documents/git-repos/amazon/analyses/T7-incubations/unipept/

/home/millieginty/Documents/git-repos/amazon/analyses/T7-incubations/unipept


In [5]:
# read the data into pandas dataframes
MN_T24_GF_318a = pd.read_csv("lca/cleaned/318_MN_T24_GF_lca.csv")

# delete any Metazoa hits because they're trypsin
MN_T24_GF_318b = MN_T24_GF_318a[MN_T24_GF_318a.kingdom != 'Metazoa']

# keep only entries to the phylum level
MN_T24_GF_318 = MN_T24_GF_318b[MN_T24_GF_318b['phylum'].notnull()]

# How many peptides to the phylum level?
print('# of phylum peptides = ', len(MN_T24_GF_318))

MN_T24_GF_318.head(6)

# of phylum peptides =  39


,peptide,lca,superkingdom,kingdom,subkingdom,superphylum,phylum,subphylum,superclass,class,...,tribe,subtribe,genus,subgenus,species group,species subgroup,species,subspecies,varietas,forma
1,LATVLSPR,Gammaproteobacteria,Bacteria,NaN,NaN,NaN,Proteobacteria,NaN,NaN,Gammaproteobacteria,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,VATVSPLR,Solirubrobacteraceae,Bacteria,NaN,NaN,NaN,Actinobacteria,NaN,NaN,Thermoleophilia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30,VGHWLPR,Streptoalloteichus hindustanus,Bacteria,NaN,NaN,NaN,Actinobacteria,NaN,NaN,Actinomycetia,...,Streptoalloteichus,NaN,NaN,NaN,Streptoalloteichus hindustanus,NaN,NaN,NaN,NaN,NaN
80,TLESHFEK,Candidatus Falkowbacteria,Bacteria,NaN,NaN,NaN,Candidatus Falkowbacteria,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,LEHVEYK,Thermofilum,Archaea,NaN,NaN,NaN,Crenarchaeota,NaN,NaN,Thermoprotei,...,Thermofilum,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
109,TMNDDSEK,Paraburkholderia,Bacteria,NaN,NaN,NaN,Proteobacteria,NaN,NaN,Betaproteobacteria,...,Paraburkholderia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Now I want to read in the file containing the stripped peptides with NAAF values
####  - NAAF stands for 'noramlized area abunace factor'

### I want to join the dataframes if they share an index (stripped peptide with equated leucine and isoleucines)
#### - That means I'll reindex the processed peptide file

In [6]:
MN_T24_GF_318_NAAFa = pd.read_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/I-L_NAAFs/318A_MN_T24_GF_DN50_ILnaafs.csv")
MN_T24_GF_318_NAAFb = pd.read_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/I-L_NAAFs/318B_MN_T24_GF_DN50_ILnaafs.csv")
MN_T24_GF_318_NAAFc = pd.read_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/I-L_NAAFs/318C_MN_T24_GF_DN50_ILnaafs.csv")


frames = [MN_T24_GF_318_NAAFa, MN_T24_GF_318_NAAFb]

MN_T24_GF_318_NAAF = pd.concat(frames, sort=False)

MN_T24_GF_318_NAAF.set_index('stripped_peptide')
MN_T24_GF_318_NAAF = MN_T24_GF_318_NAAF.loc[:, ~MN_T24_GF_318_NAAF.columns.str.contains('^Unnamed')]

MN_T24_GF_318_NAAF.rename(columns = {'stripped_peptide':'peptide'}, inplace = True)

print('# of total peptides = ', len(MN_T24_GF_318_NAAF))

print('column names:', MN_T24_GF_318_NAAF.columns)

MN_T24_GF_318_NAAF.head()

# of total peptides =  354
column names: Index(['peptide', 'Area', 'NAAF_num.'], dtype='object')


,peptide,Area,NAAF_num.
0,LSSPATLNSR,79000000.0,7900000.0
1,LSSPATLNSR,79000000.0,7900000.0
2,LSSPATLNSR,79000000.0,7900000.0
3,LATVLSPR,50600000.0,6325000.0
4,LATVLSPR,50600000.0,6325000.0


In [7]:
# get the intersection of the phylum peptides and corresponding peptides w/ NAAFs

over = np.intersect1d(MN_T24_GF_318["peptide"], MN_T24_GF_318_NAAF["peptide"])

print(over)

['CHHEEK' 'LATVLSPR' 'LATVTVPR' 'SLSNSNER' 'SPWALALR' 'TLESHFEK'
 'TMNDDSEK' 'TTTVSLPR' 'VATVSPLR' 'VGHWLPR']


In [8]:
# join the dataframes if the peptide values are the same using 'join'
# since a couple are de novo only (more for bacteria), we won't have all the UniPept peptides overlap 

MN_T24_GF_318.set_index('peptide', inplace=True)
MN_T24_GF_318_NAAF.set_index('peptide', inplace=True)

MN_T24_GF_318_Phy = MN_T24_GF_318.join(MN_T24_GF_318_NAAF, how='left', rsuffix='_other')


print('# of total phylum-level peptides = ', len(MN_T24_GF_318_Phy))

MN_T24_GF_318_Phy.head()

# of total phylum-level peptides =  57


,lca,superkingdom,kingdom,subkingdom,superphylum,phylum,subphylum,superclass,class,subclass,...,genus,subgenus,species group,species subgroup,species,subspecies,varietas,forma,Area,NAAF_num.
peptide,,,,,,,,,,,,,,,,,,,,,
AESDSLER,Pseudogymnoascus,Eukaryota,Fungi,Dikarya,NaN,Ascomycota,Pezizomycotina,NaN,Leotiomycetes,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ALTVVDPR,Opitutae,Bacteria,NaN,NaN,NaN,Verrucomicrobia,NaN,NaN,Opitutae,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AMTETNYNK,Vibrio harveyi group,Bacteria,NaN,NaN,NaN,Proteobacteria,NaN,NaN,Gammaproteobacteria,NaN,...,Vibrio harveyi group,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CHHEEK,Aspergillus,Eukaryota,Fungi,Dikarya,NaN,Ascomycota,Pezizomycotina,NaN,Eurotiomycetes,Eurotiomycetidae,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,173000.0,28833.333333
CNLSGWK,Candidatus Uhrbacteria,Bacteria,NaN,NaN,NaN,Candidatus Uhrbacteria,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# write to a csv

MN_T24_GF_318_Phy.to_csv("lca/NAAF/MN_T24_GF_318_Phy_naaf.csv")